In [77]:
# dependencies

# basic
from datetime import datetime
import json
import pandas as pd
from pandas import ExcelFile
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np
import requests
import seaborn as sns
import time

# nba api
import nba_api
from nba_api.stats.endpoints import BoxScoreAdvancedV2, BoxScoreDefensive, BoxScoreMiscV2, BoxScorePlayerTrackV2, BoxScoreTraditionalV2, BoxScoreUsageV2
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder, leaguegamelog, playergamelog

# sklearn
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, Normalizer, StandardScaler
from sklearn.svm import SVC

# tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

In [8]:
nbateams = teams.get_teams()

In [30]:
df_current_season_games_full = leaguegamelog.LeagueGameLog().get_data_frames()[0]

In [31]:
df_current_season_games_full

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22019,1610612746,LAC,LA Clippers,0021900002,2019-10-22,LAC vs. LAL,W,240,42,...,34,45,24,8,5,14,25,112,10,1
1,22019,1610612747,LAL,Los Angeles Lakers,0021900002,2019-10-22,LAL @ LAC,L,240,37,...,32,41,20,4,7,15,24,102,-10,1
2,22019,1610612761,TOR,Toronto Raptors,0021900001,2019-10-22,TOR vs. NOP,W,265,42,...,41,57,23,7,3,17,24,130,8,1
3,22019,1610612740,NOP,New Orleans Pelicans,0021900001,2019-10-22,NOP @ TOR,L,265,43,...,37,53,30,4,9,19,34,122,-8,1
4,22019,1610612755,PHI,Philadelphia 76ers,0021900008,2019-10-23,PHI vs. BOS,W,240,37,...,50,62,24,6,7,17,34,107,14,1
5,22019,1610612738,BOS,Boston Celtics,0021900008,2019-10-23,BOS @ PHI,L,240,33,...,31,41,18,4,2,11,29,93,-14,1
6,22019,1610612752,NYK,New York Knicks,0021900010,2019-10-23,NYK @ SAS,L,240,44,...,34,39,24,16,1,14,32,111,-9,1
7,22019,1610612759,SAS,San Antonio Spurs,0021900010,2019-10-23,SAS vs. NYK,W,240,42,...,41,51,22,6,5,21,18,120,9,1
8,22019,1610612758,SAC,Sacramento Kings,0021900012,2019-10-23,SAC @ PHX,L,240,34,...,35,47,17,5,5,27,24,95,-29,1
9,22019,1610612756,PHX,Phoenix Suns,0021900012,2019-10-23,PHX vs. SAC,W,240,49,...,35,44,31,13,9,14,25,124,29,1


In [29]:
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result

In [32]:
df_current_season_games = combine_team_games(df_current_season_games_full)

In [36]:
BoxScoreAdvancedV2(game_id = '0021900011').get_data_frames()[0]
# BoxScoreDefensive
# BoxScoreMiscV2
# BoxScorePlayerTrackV2
# BoxScoreUsageV2

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,E_OFF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0021900011,1610612760,OKC,Oklahoma City,1628390,Terrance Ferguson,F,,23:51,81.0,...,33.3,0.000,1.136,0.036,0.037,96.76,96.60,80.50,48,0.023
1,0021900011,1610612760,OKC,Oklahoma City,201568,Danilo Gallinari,F,,28:20,115.6,...,5.6,0.615,0.691,0.250,0.251,97.60,98.28,81.90,59,0.224
2,0021900011,1610612760,OKC,Oklahoma City,203500,Steven Adams,C,,27:42,96.1,...,13.3,0.125,0.141,0.206,0.209,97.49,97.04,80.87,56,-0.037
3,0021900011,1610612760,OKC,Oklahoma City,1628983,Shai Gilgeous-Alexander,G,,36:46,96.8,...,3.6,0.500,0.507,0.325,0.332,99.61,101.18,84.32,77,0.094
4,0021900011,1610612760,OKC,Oklahoma City,101108,Chris Paul,G,,30:07,93.4,...,4.8,0.600,0.656,0.243,0.249,101.27,103.60,86.33,65,0.266
5,0021900011,1610612760,OKC,Oklahoma City,203471,Dennis Schroder,,,30:40,87.0,...,14.3,0.188,0.188,0.133,0.135,106.12,105.65,88.04,67,0.020
6,0021900011,1610612760,OKC,Oklahoma City,203457,Nerlens Noel,,,4:42,36.4,...,0.0,1.000,1.000,0.091,0.091,107.27,102.16,85.14,10,0.409
7,0021900011,1610612760,OKC,Oklahoma City,1629647,Darius Bazley,,,19:15,57.8,...,25.0,0.000,0.000,0.082,0.083,106.57,104.73,87.27,42,-0.098
8,0021900011,1610612760,OKC,Oklahoma City,1628977,Hamidou Diallo,,,23:01,86.1,...,25.0,0.625,0.615,0.203,0.208,110.12,112.58,93.82,53,0.080
9,0021900011,1610612760,OKC,Oklahoma City,203488,Mike Muscala,,,15:36,96.3,...,11.1,0.000,0.347,0.175,0.178,109.10,110.76,92.30,36,0.080


In [46]:
test_array = df_current_season_games['GAME_ID'].unique()[:10]
unique_array = df_current_season_games['GAME_ID'].unique()
test_df = df_current_season_games_full.copy()

In [ ]:
df_adv_list = []
df_def_list = []
df_misc_list = []
df_tracking_list = []
df_usage_list = []
df_traditional_list = []

startTime = datetime.now()

game_ok_count = 0
game_no_count = 0

game_problem_list = []

for game in test_array:
  
    try:

        print(f'Searching for game: {game}...')

        df_advanced = BoxScoreAdvancedV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 1.Advanced df found and saved')
        
        df_defensive = BoxScoreDefensive(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 2.Defensive df found and saved')

        df_misc = BoxScoreMiscV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 3.Misc df found and saved')

        df_tracking = BoxScorePlayerTrackV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 4.Tracking df found and saved')

        df_usage = BoxScoreUsageV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 5.Usage df found and saved')
        
        df_traditional = BoxScoreTraditionalV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 6.Traditional df found and saved')

#         x, = np.where(unique_array == game)
        x, = np.where(test_array == game)
#         x, = np.where(missing_games == game)

        game_ok_count += 1

        print(f'--- {x+1} games searched ---')
        print(f'--- {game_ok_count} games found ---')

        time.sleep(5)

        df_adv_list.append(df_advanced)
        df_def_list.append(df_defensive)
        df_misc_list.append(df_misc)
        df_tracking_list.append(df_tracking)
        df_usage_list.append(df_usage)
        df_traditional_list.append(df_traditional)

        print(f'--- {game_ok_count} games appended ---')

        print(f'--- Time: {datetime.now() - startTime} ---')

    except:
    
    #   problem list
        game_no_count += 1

        game_problem_list.append(game)

        print(f'game {game} encountered timeout')
        print(f'--- {game_no_count} games with problems ---')

    #   sleep
        print(f'--- sleeping for 300 seconds ---')
        time.sleep(300)

    #   try again to search
        try:

            print(f'Searching again for game: {game}...')

            df_advanced = BoxScoreAdvancedV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 1.Advanced df found and saved')
        
            df_defensive = BoxScoreDefensive(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 2.Defensive df found and saved')

            df_misc = BoxScoreMiscV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 3.Misc df found and saved')

            df_tracking = BoxScorePlayerTrackV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 4.Tracking df found and saved')

            df_usage = BoxScoreUsageV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 5.Usage df found and saved')
            
            df_traditional = BoxScoreTraditionalV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 6.Traditional df found and saved')

            x, = np.where(unique_array == game)

            game_ok_count += 1

            print(f'--- {x+1} games searched ---')
            print(f'--- {game_ok_count} games found ---')

            time.sleep(5)

            df_adv_list.append(df_advanced)
            df_def_list.append(df_defensive)
            df_misc_list.append(df_misc)
            df_tracking_list.append(df_tracking)
            df_usage_list.append(df_usage)
            df_traditional_list.append(df_traditional)

            print(f'--- {game_ok_count} games appended ---')

            print(f'--- Time: {datetime.now() - startTime} ---')

        except:

            print(f'game {game} encountered another timeout')
            print(f'--- {game_no_count} games with problems ---')

        #   sleep
            print(f'--- sleeping for 300 seconds ---')
            time.sleep(300)

            continue

    continue

df_adv_all = pd.concat(df_adv_list)
df_def_all = pd.concat(df_def_list)
df_misc_all = pd.concat(df_misc_list)
df_tracking_all = pd.concat(df_tracking_list)
df_usage_all = pd.concat(df_usage_list)
df_traditional_all = pd.concat(df_traditional_list)

# df_def_missing = pd.concat(df_def_list)
# df_misc_missing = pd.concat(df_misc_list)
# df_tracking_missing = pd.concat(df_tracking_list)
# df_usage_missing = pd.concat(df_usage_list)

print(f'--- merged df ---')

Searching for game: 0021900002...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [1] games searched ---
--- 1 games found ---
--- 1 games appended ---
--- Time: 0:00:29.355504 ---
Searching for game: 0021900001...


In [40]:
df_adv_all.to_csv('current_advanced.csv')
df_def_all.to_csv('current_defensive.csv')
df_misc_all.to_csv('current_misc.csv')
df_tracking_all.to_csv('current_tracking.csv')
df_usage_all.to_csv('current_usage.csv')
df_traditional_all.to_csv('current_traditional.csv')

In [71]:
stats_df = df_def_all.merge(df_adv_all, 
                            on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [72]:
# stats_df = test_df.merge(df_def_all, on=['GAME_ID','PLAYER_ID','TEAM_ID'], how='outer')
stats_df2 = stats_df.merge(df_misc_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [73]:
# stats_df = test_df.merge(df_misc_all, on=['GAME_ID','PLAYER_ID','TEAM_ID'], how='outer')
stats_df3 = stats_df2.merge(df_tracking_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [74]:
# stats_df = test_df.merge(df_tracking_all, on=['GAME_ID','PLAYER_ID','TEAM_ID'], how='outer')
stats_df4 = stats_df3.merge(df_usage_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [75]:
stats_df5 = stats_df4.merge(df_traditional_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,TEAM_NICKNAME,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MATCHUP_MIN,...,PCT_DREB,PCT_REB,PCT_AST,PCT_TOV,PCT_STL,PCT_BLK,PCT_BLKA,PCT_PF,PCT_PFD,PCT_PTS
0,0021900002,1610612747,LAL,Los Angeles,Lakers,203076,Anthony Davis,F,,12:23,...,0.207,0.257,0.313,0.231,0.333,0.286,0.333,0.136,0.500,0.301
1,0021900002,1610612747,LAL,Los Angeles,Lakers,2544,LeBron James,F,,11:55,...,0.375,0.323,0.533,0.455,0.250,0.200,0.500,0.200,0.261,0.250
2,0021900002,1610612747,LAL,Los Angeles,Lakers,201580,JaVale McGee,C,,5:20,...,0.083,0.143,0.000,0.167,0.000,0.500,0.000,0.000,0.000,0.118
3,0021900002,1610612747,LAL,Los Angeles,Lakers,201980,Danny Green,G,,10:20,...,0.250,0.241,0.000,0.000,0.667,0.167,0.000,0.167,0.235,0.373
4,0021900002,1610612747,LAL,Los Angeles,Lakers,202340,Avery Bradley,G,,7:53,...,0.200,0.176,0.000,0.222,0.000,0.000,0.000,0.250,0.000,0.174
5,0021900002,1610612747,LAL,Los Angeles,Lakers,203484,Kentavious Caldwell-Pope,,,9:12,...,0.158,0.115,0.250,0.125,0.000,0.000,0.000,0.294,0.067,0.000
6,0021900002,1610612747,LAL,Los Angeles,Lakers,2730,Dwight Howard,,,6:14,...,0.273,0.353,0.125,0.000,0.000,1.000,1.000,0.400,0.300,0.065
7,0021900002,1610612747,LAL,Los Angeles,Lakers,203584,Troy Daniels,,,6:13,...,0.000,0.000,0.200,0.200,0.000,0.000,0.000,0.200,0.111,0.222
8,0021900002,1610612747,LAL,Los Angeles,Lakers,1626188,Quinn Cook,,,5:20,...,0.083,0.059,0.333,0.200,0.000,0.000,0.000,0.000,0.000,0.100
9,0021900002,1610612747,LAL,Los Angeles,Lakers,201162,Jared Dudley,,,4:17,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.667,0.000,0.286


In [76]:
stats_df5.keys()

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'TEAM_NICKNAME',
       'PLAYER_ID', 'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MATCHUP_MIN',
       'PARTIAL_POSS', 'SWITCHES_ON', 'PLAYER_PTS', 'DREB', 'MATCHUP_AST',
       'MATCHUP_TOV', 'STL', 'BLK_x', 'MATCHUP_FGM', 'MATCHUP_FGA',
       'MATCHUP_FG_PCT', 'MATCHUP_FG3M', 'MATCHUP_FG3A', 'MATCHUP_FG3_PCT',
       'MIN_x', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT',
       'USG_PCT_x', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE',
       'MIN_y', 'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT',
       'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT',
       'BLK_y', 'BLKA', 'PF', 'PFD', 'MIN_x', 'SPD', 'DIST', 'ORBC', 'DRBC',
       'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM', 'CFGA',
       'CFG_PCT', '